In [6]:
import json
import re
from openai import OpenAI
from tqdm import tqdm
import os
import re
import random

# key = ""

# client = OpenAI(api_key= key)
client = OpenAI(
    base_url='http://localhost:11434/v1/',

    # required but ignored
    api_key='ollama',
)

def write_jsonl_file(file_path, dict_list):
    """
    Write a list of dictionaries to a JSON Lines file.

    Args:
    - file_path (str): The path to the file where the data will be written.
    - dict_list (list): A list of dictionaries to write to the file.
    """
    with open(file_path, 'w') as file:
        for dictionary in dict_list:
            # Convert the dictionary to a JSON string and write it to the file.
            json_line = json.dumps(dictionary)
            file.write(json_line + '\n')

def read_jsonl_file(file_path):
    """
    Parses a JSONL (JSON Lines) file and returns a list of dictionaries.

    Args:
        file_path (str): The path to the JSONL file to be read.

    Returns:
        list of dict: A list where each element is a dictionary representing
            a JSON object from the file.
    """
    jsonl_lines = []
    with open(file_path, 'r', encoding="utf-8") as file:
        for line in file:
            json_object = json.loads(line)
            jsonl_lines.append(json_object)
            
    return jsonl_lines

def create_query(item):
    """
    Creates the input for the model using the question and the multiple choice options.

    Args:
        item (dict): A dictionary containing the question and options.
            Expected keys are "question" and "options", where "options" is another
            dictionary with keys "A", "B", "C", and "D".

    Returns:
        str: A formatted query combining the question and options, ready for use.
    """
    query = f"""## Question {item["question"]}
A. {item["options"]["A"]}             
B. {item["options"]["B"]}
C. {item["options"]["C"]}
D. {item["options"]["D"]}"""
    
    return query

def build_zero_shot_prompt(system_prompt, question):
    """
    Builds the zero-shot prompt.

    Args:
        system_prompt (str): Task Instruction for the LLM
        content (dict): The content for which to create a query, formatted as
            required by `create_query`.

    Returns:
        list of dict: A list of messages, including a system message defining
            the task and a user message with the input question.
    """
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": create_query(question)}]
    return messages

def format_answer(cot, answer):
    return f"""## Answer
{cot}
Therefore, the answer is {answer}"""

def build_few_shot_prompt(system_prompt, question, examples, include_cot=True):
    """
    Builds the few-shot prompt.

    Args:
        system_prompt (str): Task Instruction for the LLM
        content (dict): The content for which to create a query, formatted as
            required by `create_query`.

    Returns:
        list of dict: A list of messages, including a system message defining
            the task and a user message with the input question.
    """
    messages = [{"role": "system", "content": system_prompt}]
    
    for elem in examples:
        messages.append({"role": "user", "content": create_query(elem)})
        if include_cot:
            messages.append({"role": "assistant", "content": format_answer(elem["cot"], elem["answer_idx"])})        
        else:           
            answer_string = f"""## Answer\nTherefore, the answer is {elem["answer_idx"]}"""
            messages.append({"role": "assistant", "content": answer_string})
            
    messages.append({"role": "user", "content": create_query(question)})
    return messages

def get_response(messages, model_name, temperature = 0.0, max_tokens = 10):
    """
    Obtains the responses/answers of the model through the chat-completions API.

    Args:
        messages (list of dict): The built messages provided to the API.
        model_name (str): Name of the model to access through the API
        temperature (float): A value between 0 and 1 that controls the randomness of the output.
        A temperature value of 0 ideally makes the model pick the most likely token, making the outputs deterministic.
        max_tokens (int): Maximum number of tokens that the model should generate

    Returns:
        str: The response message content from the model.
    """
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def matches_ans_option(s):
    """
    Checks if the string starts with the specific pattern 'Therefore, the answer is [A-Z]'.
    
    Args:
    s (str): The string to be checked.

    Returns:
    bool: True if the string matches the pattern, False otherwise.
    """
    return bool(re.match(r'^Therefore, the answer is [A-Z]', s))

def extract_ans_option(s):
    """
    Extracts the answer option (a single capital letter) from the start of the string.
    
    Args:
    s (str): The string containing the answer pattern.

    Returns:
    str or None: The captured answer option if the pattern is found, otherwise None.
    """
    match = re.search(r'^Therefore, the answer is ([A-Z])', s)
    if match:
        return match.group(1)  # Returns the captured alphabet
    return None 

def matches_answer_start(s):
    """
    Checks if the string starts with the markdown header '## Answer'.
    
    Args:
    s (str): The string to be checked.

    Returns:
    bool: True if the string starts with '## Answer', False otherwise.
    """
    return s.startswith("## Answer")

def validate_response(s):
    """
    Validates a multi-line string response that it starts with '## Answer' and ends with the answer pattern.
    
    Args:
    s (str): The multi-line string response to be validated.

    Returns:
    bool: True if the response is valid, False otherwise.
    """
    file_content = s.split("\n")
    
    return matches_ans_option(file_content[-1]) and matches_answer_start(s)

def parse_answer(response):
    """
    Parses a response that starts with '## Answer', extracting the reasoning and the answer choice.
    
    Args:
    response (str): The multi-line string response containing the answer and reasoning.

    Returns:
    tuple: A tuple containing the extracted CoT reasoning and the answer choice.
    """
    split_response = response.split("\n")
    assert split_response[0] == "## Answer"
    cot_reasoning = "\n".join(split_response[1:-1]).strip()
    ans_choice = extract_ans_option(split_response[-1])
    return cot_reasoning, ans_choice


system_prompt = """You are an expert medical professional. You are provided with a medical question with multiple answer choices.
Your goal is to think through the question carefully and explain your reasoning step by step before selecting the final answer.
Respond only with the reasoning steps and answer as specified below.
Below is the format for each question and answer:

Input:
## Question: {{question}}
{{answer_choices}}

Output:
## Answer
(model generated chain of thought explanation)
Therefore, the answer is [final model answer (e.g. A,B,C,D)]"""

system_zero_shot_prompt = """You are an expert medical professional. You are provided with a medical question with multiple answer choices.
Your goal is to think through the question carefully and respond directly with the answer option.
Below is the format for each question and answer:

Input:
## Question: {{question}}
{{answer_choices}}

Output:
## Answer
Therefore, the answer is [final model answer (e.g. A,B,C,D)]"""

def get_embedding(text, model="all-minilm:22m"):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

## Generate chain of thoughts embeddings for datapoints in the development set

In [11]:
train_data = read_jsonl_file("data/phrases_no_exclude_train.jsonl")
train_data = train_data[:2000]

In [12]:
cot_responses = []

if not os.path.isdir("cot_responses"):
    os.mkdir("cot_responses")

for idx, item in enumerate(tqdm(train_data)):
    curr_fn = os.path.join("cot_responses", str(idx) + ".txt")
    if not os.path.exists(curr_fn):
        prompt = build_zero_shot_prompt(system_prompt, item)
        try:
            response = get_response(prompt, model_name="llama3.2:1b", max_tokens=500)
            cot_responses.append(response)
            with open(curr_fn, "w", encoding="utf-8") as f:
                f.write(response)           
        except Exception as e :
            print(str(e))
            cot_responses.append("")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [2:41:50<00:00,  4.86s/it]


In [13]:
questions_dict = []
ctr = 0
for idx, question in enumerate(tqdm(train_data)):
    file =  open(os.path.join("cot_responses", str(idx) + ".txt"), encoding="utf-8").read()
    if not validate_response(file):
        continue
    
    cot, pred_ans = parse_answer(file)
    
    dict_elem = {}
    dict_elem["idx"] = idx
    dict_elem["question"] = question["question"]
    dict_elem["answer"] = question["answer"]
    dict_elem["options"] = question["options"]
    dict_elem["cot"] = cot
    dict_elem["pred_ans"] = pred_ans
    questions_dict.append(dict_elem)        



100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 6558.04it/s]


In [14]:
if not os.path.isdir("tmp_data"):
    os.mkdir("tmp_data")

write_jsonl_file(os.path.join("tmp_data", "cot_responses_medqa_train_set.jsonl"), questions_dict)

In [ ]:
questions_dict = read_jsonl_file(os.path.join("tmp_data", "cot_responses_medqa_train_set.jsonl"))

## Filter questions whose predicted answer does not match the actual answer

In [15]:
filtered_questions_dict = []
for item in tqdm(questions_dict):
    pred_ans = item["options"][item["pred_ans"]]
    if pred_ans == item["answer"]:
        filtered_questions_dict.append(item)
print("No. of Raw Responses: ", len(questions_dict))
print("No. of Good Responses: ", len(filtered_questions_dict))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 1143901.09it/s]

No. of Raw Responses:  240
No. of Good Responses:  98


## Embed all questions

In [16]:
for item in tqdm(filtered_questions_dict):
    item["embedding"] = get_embedding(item["question"])
    inv_options_map = {v:k for k,v in item["options"].items()}
    item["answer_idx"] = inv_options_map[item["answer"]]    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:04<00:00, 22.43it/s]


In [17]:
write_jsonl_file(os.path.join("tmp_data","cot_responses_medqa_train_set_filtered_with_embeddings.jsonl"), filtered_questions_dict)

## Fit K-NN

In [ ]:
filtered_questions_dict = read_jsonl_file(os.path.join("tmp_data","cot_responses_medqa_train_set_filtered_with_embeddings.jsonl")

In [18]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Extract embeddings and keep track of indices
embeddings = np.array([d["embedding"] for d in filtered_questions_dict])
indices = list(range(len(filtered_questions_dict))) # No need

# Train KNN model
knn = NearestNeighbors(n_neighbors=3, algorithm='auto', metric='cosine').fit(embeddings)

## Implement inference pipeline

In [20]:
test_samples = read_jsonl_file("data/phrases_no_exclude_test.jsonl")
print("test set size: ", len(test_samples))

test set size:  1273


In [ ]:
def shuffle_option_labels(answer_options):
    """
    Shuffles the options of the question.
    
    Parameters:
    answer_options (dict): A dictionary with the options.

    Returns:
    dict: A new dictionary with the shuffled options.
    """
    options = list(answer_options.values())
    random.shuffle(options)
    labels = [chr(i) for i in range(ord('A'), ord('A') + len(options))]
    shuffled_options_dict = {label: option for label, option in zip(labels, options)}
    
    return shuffled_options_dict


for question in tqdm(test_samples, colour ="green"):
    question_variants = []
    prompt_variants = []
    cot_responses = []
    question_embedding = get_embedding(question["question"])
    distances, top_k_indices = knn.kneighbors([question_embedding], n_neighbors=5)
    top_k_dicts = [filtered_questions_dict[i] for i in top_k_indices[0]]
    question["outputs"] = []
    
    for idx in range(5):
        question_copy = question.copy()
        shuffled_options = shuffle_option_labels(question["options"])
        inv_map = {v:k for k,v in shuffled_options.items()}
        
        question_copy["options"] = shuffled_options
        question_copy["answer_idx"] = inv_map[question_copy["answer"]]
        question_variants.append(question_copy)
        prompt = build_few_shot_prompt(system_prompt,  question_copy, top_k_dicts)
        prompt_variants.append(prompt)
    
    for prompt in tqdm(prompt_variants):
        response = get_response(prompt, model_name="llama3.2:1b", max_tokens=500)
        cot_responses.append(response)
    
    for question_sample, answer in zip(question_variants, cot_responses):
        if validate_response(answer):
            cot, pred_ans = parse_answer(answer)
            
        else:
            cot = ""
            pred_ans = ""
                
        question["outputs"].append({"question": question_sample["question"], "options": question_sample["options"], "cot": cot, "pred_ans": question_sample["options"].get(pred_ans, "")})
    

 80%|████████████████████████████████████████████████████████████████████████████████████▊                     | 4/5 [00:31<00:07,  7.90s/it]


 60%|███████████████████████████████████████████████████████████████▌                                          | 3/5 [00:22<00:15,  7.53s/it]


 40%|██████████████████████████████████████████▍                                                               | 2/5 [00:15<00:22,  7.64s/it]


 20%|█████████████████████▏                                                                                    | 1/5 [00:09<00:36,  9.14s/it]


  0%|                                                                                                                  | 0/5 [00:00<?, ?it/s]


...

 20%|█████████████████████▏                                                                                    | 1/5 [00:12<00:50, 12.65s/it]


  0%|                                                                                                                  | 0/5 [00:00<?, ?it/s]


 68%|█████████████████████████████████████████████████████████████████▊                              | 872/1273 [11:14:37<6:33:13, 58.84s/it]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:07<00:00, 13.58s/it]


  0%|                                                                                                                  | 0/5 [00:00<?, ?it/s]

In [ ]:
write_jsonl_file(os.path.join("tmp_data","final_processed_test_set_responses_medprompt.jsonl"), test_samples )

In [ ]:
test_samples = read_jsonl_file(os.path.join("tmp_data","final_processed_test_set_responses_medprompt.jsonl"))

In [ ]:
from collections import Counter

def find_mode_string_list(string_list):
    """
    Finds the most frequently occurring strings.

    Parameters:
    string_list (list of str): A list of strings.
    Returns:
    list of str or None: A list containing the most frequent string(s) from the input list.
                         Returns None if the input list is empty.
    """    
    if not string_list:
        return None  

    string_counts = Counter(string_list)
    max_freq = max(string_counts.values())
    mode_strings = [string for string, count in string_counts.items() if count == max_freq]
    return mode_strings

In [ ]:
ctr = 0 

for idx,item in enumerate(test_samples):
    print(idx)
    pred_ans = [x["pred_ans"] for x in item["outputs"]]
    freq_ans = find_mode_string_list(pred_ans)
    
    if len(freq_ans) > 1:
        final_prediction = ""
    
    else:
        final_prediction = freq_ans[0]
        
    if final_prediction == item["answer"]:
        ctr +=1

## Zero shot evaluation

In [ ]:
zero_shot_predictions = []

for item in tqdm(test_samples):
    messages = build_zero_shot_prompt(system_zero_shot_prompt, item)
    response = get_response(messages, "llama3.2:1b", max_tokens=50)
    pred_option = extract_ans_option(response.split("\n")[-1])
    zero_shot_predictions.append(pred_option)

In [ ]:
ctr =0 
for gt,pred in zip(test_samples, zero_shot_predictions):
    if pred == gt["answer_idx"]:
        ctr +=1

In [ ]:
ctr / 500

In [ ]:
with open("zero_shot_predictions.json", "w") as f:
    f.write(json.dumps({"predictions": zero_shot_predictions}))

In [ ]:
zero_shot_predictions = json.loads(open("zero_shot_predictions.json").read())["predictions"]

## Random few-shot evaluation

In [ ]:
few_shot_examples = random.sample(filtered_questions_dict, 5)

In [ ]:
few_shot_predictions = []
for item in tqdm(test_samples):
    messages = build_few_shot_prompt(system_zero_shot_prompt, item, few_shot_examples, include_cot=False)
    response = get_response(messages, "llama3.2:1b", max_tokens=50)
    pred_option = extract_ans_option(response.split("\n")[-1])
    few_shot_predictions.append(pred_option)

In [ ]:
ctr =0 
for gt,pred in zip(test_samples, few_shot_predictions):
    if pred == gt["answer_idx"]:
        ctr +=1

In [ ]:
ctr / 500

In [ ]:
with open("few_shot_predictions.json", "w") as f:
    f.write(json.dumps({"predictions": few_shot_predictions}))

In [ ]:
with open("few_shot_examples.json", "w") as f:
    f.write(json.dumps(few_shot_examples))

## Random few-shot evaluation with CoT

In [ ]:
few_shot_cot_predictions = []
for item in tqdm(test_samples):
#     messages = build_few_shot_prompt(system_zero_shot_prompt, item, few_shot_examples, include_cot=True)
    messages = build_few_shot_prompt(system_prompt, item, few_shot_examples, include_cot=True)
    answer = get_response(messages, "llama3.2:1b", max_tokens=500)
    if validate_response(answer):
        cot, pred_ans = parse_answer(answer)
    else:
        cot = ""
        pred_ans = ""
    few_shot_cot_predictions.append({"cot": cot, "pred_ans": pred_ans})

In [ ]:
ctr =0 
for gt,pred in zip(test_samples, few_shot_cot_predictions):
    if pred["pred_ans"] == gt["answer_idx"]:
        ctr +=1

In [ ]:
with open("few_shot_predictions_cot.json", "w") as f:
    f.write(json.dumps({"predictions": few_shot_cot_predictions}))